In [2]:
import os
import mne
import pickle
import itertools
import matplotlib.pyplot as plt
import numpy as np
import colorsys
import sys
sys.path.append("/../../MFRS")
from utils.config import similarity_folder, networks, sensors_position, mask_params, channels_grad2, channels_grad1, channels_mag
from plot_utils import get_layers_similarity, get_bootstrap_values, extract_layers_max_sim_values
from plot_functions import plot_MEG_topomaps

Opening raw data file /home/hamza97/scratch/data/MFRS_data/ds000117/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_meg.fif...
    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle
    Range : 248600 ... 788699 =    226.000 ...   716.999 secs
Ready.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306

In [63]:
model_names = ["FaceNet", "SphereFace", "resnet50", "cornet_s"]
stimuli_file_names = ["Fam", "Unfam"]
activ_types = ["trained", "untrained"]
# Define a dictionary to map model names to indices
model_indices = {
    "SphereFace": 0,
    "FaceNet": 1,
    "resnet50": 2,
    "cornet_s": 3
}

# Define a nested dictionary to map combinations of activ_type and stimuli_file_name to offsets
combination_offsets = {
    ("trained", "Fam"): 0,
    ("trained", "Unfam"): 4,
    ("untrained", "Unfam"): 12,
    ("untrained", "Fam"): 8
}
combinations = list(itertools.product(model_names, stimuli_file_names, activ_types))

In [64]:
results = np.zeros( (3, 16))
error_bars = np.zeros( (3, 16))

accuracy = [87.066, 90.679, 91.856, 75.908]
for combination in combinations:
    model_name, stimuli_file_name, activ_type = combination
    file = os.path.join(similarity_folder, f"{model_name}_{stimuli_file_name}_model_sim_scores_{activ_type}_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)
    layer_similarities, extremum_values = get_layers_similarity(data, ["model"], correlation_measure="pearson", epsilon=0.05)
    i = model_indices.get(model_name)  
    
    i += combination_offsets.get((activ_type, stimuli_file_name))

    results[0][i] = max(layer_similarities["model"][0])
    results[1][i] = max(layer_similarities["model"][1])
    results[2][i] = max(layer_similarities["model"][2])

    file = os.path.join(similarity_folder, f"{model_name}_{stimuli_file_name}_model_bootstrap_scores_{activ_type}_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)

    semdata = get_bootstrap_values(data)
    error_bars[0][i] = semdata["model"]["mag"]
    error_bars[1][i] = semdata["model"]["grad1"]
    error_bars[2][i] = semdata["model"]["grad2"]


In [ ]:
# Original colors from the Plasma color map
original_colors = [
    "#0079FF", "#00DFA2", "#F6FA70", "#FF0060",
    "#0079FF", "#00DFA2", "#F6FA70", "#FF0060",
    "#0079FF", "#00DFA2", "#F6FA70", "#FF0060",
    "#0079FF", "#00DFA2", "#F6FA70", "#FF0060",
]

# Define the desired lightness value (between 0 and 1)
lightness = 0.7

# Generate lighter shades of colors
lighter_colors = []
for color in original_colors:
    r, g, b = colorsys.rgb_to_hls(*tuple(int(color[i:i+2], 16) / 255.0 for i in (1, 3, 5)))
    r, g, b = colorsys.hls_to_rgb(r, lightness, b)
    lighter_colors.append('#%02x%02x%02x' % tuple(int(c * 255) for c in (r, g, b)))

# Use the lighter colors in the plot
block_colors = lighter_colors

# Define the titles for each block
block_titles = ['Magnometers', 'Gradiometers 1', 'Gradiometers 2', 'Accuracy']

# Create the figure and subplots
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(8, 16))

# Plot each block
for i, ax in enumerate(axes):
    ax.set_title(block_titles[i], fontsize=14)
    if i != 3:
        ax.set_ylim(0.01, 0.28)  # Adjust the y-axis limits as per data range

        # Calculate total width required for each block
        block_width = 0.8
        num_bars = 16
        bar_width = block_width / num_bars
        error_values = error_bars[i] 
        for j in range(16):
            x = j * bar_width + (0.015 * (j // 4))  # Calculate x-coordinate for each bar
            ax.bar(x, results[i][j], color=block_colors[j], edgecolor='black', width=bar_width)
            x = j * bar_width + (0.015 * (j // 4))  # Calculate x-coordinate for each bar
            y = results[i][j]
            error = error_values[j]
            ax.errorbar(x, y, yerr=error, color='black', capsize=4, elinewidth=0.8, capthick=0.8)

            # Add dotted separation line after every 4 bars
            if (j + 1) % 4 == 0 and j != 15:
                if j == 7:
                    ax.axvline(x + bar_width - 0.0175, ymin=-0.3, ymax=1, color='black', linestyle=':', linewidth=1.0)
                else:
                    ax.axvline(x + bar_width - 0.0175, color='black', linestyle=':', linewidth=0.5)
        

        # Add agenda axis
        for v in [0.05, 0.1, 0.15, 0.2, 0.25]:
            ax.axhline(v, color='gray', linestyle='--', linewidth=0.4)
        ax.axhline(y=0.2586,linewidth=2, color='#873e23')
        ax.axhline(y=0.0176,linewidth=2, color='#e0bb41')

        ax.tick_params(axis='both', which='both', length=0, labelsize=10)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_linewidth(0.5)
        ax.spines['bottom'].set_linewidth(0.5)
        ax.set_xticks([0.075, 0.29, 0.51, 0.73])
        ax.set_xticklabels(['Familiar Input', 'Unfamiliar Input', 'Familiar Input', 'Unfamiliar Input'], fontsize=10)
        ax.text(0.25, -0.2, 'Trained models', transform=ax.transAxes, ha='center', fontsize=12)
        ax.text(0.75, -0.2, 'Untrained models', transform=ax.transAxes, ha='center', fontsize=12)
        legend_labels = ['ShpereFace', 'FaceNet', 'ResNet', 'CORnet-S', 'Upper Noise Ceiling', 'Lower Noise Ceiling']
        legend_colors = [    "#0079FF", "#00DFA2", "#F6FA70", "#FF0060", '#873e23', '#e0bb41']
        legend_patches = [plt.Rectangle((0, 0), 1 if k < 4 else 0.5, 1, fc=color) for k, color in enumerate(legend_colors)]

        ax.legend(legend_patches, legend_labels, loc='center right', fontsize=8)
    else:
        ax.set_ylim(70, 100)  # Adjust the y-axis limits as per your data range
        # Calculate total width required for each block
        num_bars = 4
        bar_width = 0.8
        
        # Plot each bar within the block
        for j in range(4):
            x = j * bar_width  + (0.015 * (j // 4)) # Calculate x-coordinate for each bar
            ax.bar(x, accuracy[j], color=block_colors[j], edgecolor='black', width=bar_width)

        ax.tick_params(axis='both', which='both', length=0, labelsize=10)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_linewidth(0.5)
        ax.spines['bottom'].set_linewidth(0.5)
        ax.set_xticks([0, 0.8, 1.6, 2.4])
        legend_labels = ['ShpereFace', 'FaceNet', 'ResNet', 'CORnet-S']

        ax.set_xticklabels(legend_labels, fontsize=10)

# Add space between subplots
plt.subplots_adjust(hspace=0.5)

fig.text(0.04, 0.6, 'Similarity Scores: Pearson Correlations', va='center', rotation='vertical', fontsize=12)
fig.text(0.04, 0.2, 'Classification Accuracy', va='center', rotation='vertical', fontsize=12)
fig.text(0.04, 0.9, 'A', va='center', fontdict=dict(fontsize=15, fontweight ='bold'))
fig.text(0.04, 0.28, 'B', va='center', fontdict=dict(fontsize=15, fontweight ='bold'))

# Display the plot
plt.show()

In [8]:
model_name = "FaceNet"
stimuli_file_names = ["Fam", "Unfam"]
activ_types = ["trained", "untrained"]
combinations = list(itertools.product(stimuli_file_names, activ_types))

In [9]:
similarity_values = [[[], [], []],
                    [[], [], []],
                    [[], [], []],
                    [[], [], []]]
combination_offsets = {
    ("trained", "Fam"): 0,
    ("trained", "Unfam"): 1,
    ("untrained", "Unfam"): 2,
    ("untrained", "Fam"): 3
}
Names = ["Fam trained", "Fam untrained", "Unfam trained", "Unfam untrained"]
for combination in combinations:
    stimuli_file_name, activ_type = combination
    file = os.path.join(similarity_folder, f"{model_name}_{stimuli_file_name}_model_sim_scores_{activ_type}_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)



    layer_similarities, extremum_values = get_layers_similarity(data, ["model"], correlation_measure="pearson", epsilon=0.05)
    i = combination_offsets.get((activ_type, stimuli_file_name))  # Get the offset from the nested dictionary

    similarity_values[i][0] = layer_similarities["model"][0]
    similarity_values[i][1] = layer_similarities["model"][1]
    similarity_values[i][2] = layer_similarities["model"][2]

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(16, 16))
for i, sim_val in enumerate(similarity_values):
    name = Names[i]
    last = i==3
    plot_MEG_topomaps(sim_val, (-0.13, 0.13), axes, i, name, fig, last)
plt.tight_layout()
fig.text(0, 1.01, 'C', va='center', fontdict=dict(fontsize=26, fontweight ='bold'))
# Set overall plot title and labels
plt.suptitle('FaceNet plots', fontsize=16, y=1.02)


In [11]:
model_names = ["FaceNet", "SphereFace", "resnet50", "cornet_s"]
stimuli_file_names = ["Fam", "Unfam"]
activ_types = ["trained", "untrained"]
# Define a dictionary to map model names to indices
model_indices = {
    "SphereFace": 0,
    "FaceNet": 1,
    "resnet50": 2,
    "cornet_s": 3
}

# Define a nested dictionary to map combinations of activ_type and stimuli_file_name to offsets
combination_offsets = {
    ("trained", "Fam"): 0,
    ("trained", "Unfam"): 1,
    ("untrained", "Unfam"): 2,
    ("untrained", "Fam"): 3
}
combinations = list(itertools.product(model_names, stimuli_file_names, activ_types))


In [ ]:
results = [np.zeros( (4, len(networks["SphereFace"]))),
            np.zeros( (4, len(networks["FaceNet"]))),
            np.zeros( (4, len(networks["resnet50"]))),
            np.zeros( (4, len(networks["cornet_s"])))]
error_bars = [np.zeros( (4, len(networks["SphereFace"]))),
            np.zeros( (4, len(networks["FaceNet"]))),
            np.zeros( (4, len(networks["resnet50"]))),
            np.zeros( (4, len(networks["cornet_s"])))]
max_idx = np.zeros((4, 4))
needed_layers = {}
masks = [[[], [], [], [],], [[], [], [], [],], [[], [], [], [],], [[], [], [], [],],]
highest_layers_sims = np.zeros( (4, 4, 102))
accuracy = [87.066, 90.679, 91.856, 75.908]
for combination in combinations:
    model_name, stimuli_file_name, activ_type = combination
    model_error_bars = np.zeros( (4, len(networks[model_name])))
    file = os.path.join(similarity_folder, f"{model_name}_{stimuli_file_name}_main_sim_scores_{activ_type}_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)
    layer_similarities, extremum_values = get_layers_similarity(data, networks[model_name])
    i = model_indices.get(model_name)  # Get the index from the dictionary, 
    values_list, max_ind, max_layer_name, mask = extract_layers_max_sim_values(layer_similarities, "grad2", channels_grad2)
    j = combination_offsets.get((activ_type, stimuli_file_name)) 
    if stimuli_file_name == "Fam" and activ_type == "trained":
        needed_layers[model_name] = max_layer_name
    results[i][j] = values_list
    masks[i][j] = mask
    max_idx[i][j] = int(max_ind)
    highest_layers_sims[j][i] = layer_similarities[networks[model_name][max_ind]][2]
    file = os.path.join(similarity_folder, f"{model_name}_{stimuli_file_name}_main_bootstrap_scores_{activ_type}_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)
    semdata = get_bootstrap_values(data)
    for k, (layer, boot_values) in enumerate(semdata.items()):
        error_bars[i][j][k] = boot_values["grad2"]

In [ ]:
labels = ["Familiar Input / Trained model", "Unfamiliar Input / Trained model", "Familiar Input / Untrained model", "Unfamiliar Input / Untrained model"]  # Add labels for the horizontal lines
subplot_titles = ["SphereFace", "FaceNet", "ResNet50", "CORnet-S"]


# Create the figure and subplots (5 rows, 4 columns)
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(30, 25))

# Plot existing data in the first row
for i, ax in enumerate(axes[0]):
    ax.set_title(subplot_titles[i], fontsize=14)
    
    # Get the number of layers for the current array
    n_layers = results[i].shape[1]

    # Define the x-axis values (layer numbers)
    x = np.arange(1, n_layers + 1)

    # Get the index of the maximum value for each layer (column) in the current array
    max_ind = np.argmax(results[i], axis=1)[0]
    colors = []
    # Plot four lines for the four rows in the current array
    for j in range(4):
        line, = ax.plot(x, results[i][j], label=labels[j])
        
        # Calculate the upper and lower bounds for the error region
        upper_bound = results[i][j] + error_bars[i][j]
        lower_bound = results[i][j] - error_bars[i][j]

        # Fill the area between the upper and lower bounds to create the shaded error region
        ax.fill_between(x, lower_bound, upper_bound, color=line.get_color(), alpha=0.3)

        # Mark the maximum value with a star marker
        ax.plot(x[int(max_idx[i][j])], results[i][j][int(max_idx[i][j])], marker='*', markersize=16, color=line.get_color(), markeredgecolor='k')
        colors.append(line.get_color())
    # Add horizontal lines to the plot
    ax.axhline(y=0.2586, linewidth=1, color='#873e23')  # Change linewidth to 1 for thin lines
    ax.axhline(y=0.0176, linewidth=1, color='#e0bb41')  # Change linewidth to 1 for thin lines

    # Set x-axis label
    ax.set_xlabel('Layer Number', fontsize=12)

    # Set y-axis label
    ax.set_ylabel('Similarity Scores: Pearson Correlations', fontsize=12)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    # Set grid
    ax.grid(True, linestyle='--', linewidth=0.5)
    ax.set_ylim(0.01, 0.26)  # Adjust the y-axis limits as per your data range

    # Add legend to each subplot
    ax.legend(loc='upper right')

# Remove the rest of the empty subplots
for row in range(1, 5):
    for col in range(4):
        # fig.delaxes(axes[row, col])
        mask_params["markerfacecolor"] = colors[row-1]
        im, _ = mne.viz.plot_topomap(highest_layers_sims[row-1, col], sensors_position, show=False, vlim=(-0.17,0.17),
                                 sphere=0.18, mask=np.array(masks[row-1][col]), mask_params=mask_params, axes=axes[row, col], extrapolate='head')
        axes[4][col].set_xlabel(subplot_titles[col], fontsize=16)
    axes[row][0].set_ylabel(labels[row-1], fontsize=16)
    fig.colorbar(im, ax=axes[row][3], orientation='vertical')

# Adjust layout
plt.tight_layout()
fig.text(0, 1.01, 'A', va='center', fontdict=dict(fontsize=26, fontweight ='bold'))
fig.text(0., 0.78, 'B', va='center', fontdict=dict(fontsize=26, fontweight ='bold'))
# Set overall plot title and labels
plt.suptitle('Grad2 plots', fontsize=16, y=1.02)

# Display the plot
plt.show()


In [14]:
model_names = ["FaceNet", "SphereFace", "resnet50", "cornet_s"]
frequency_names = [ "alpha", "beta", "theta", "delta", "Gamma", ]
# Define a dictionary to map model names to indices
model_indices = {
    "SphereFace": 0,
    "FaceNet": 1,
    "resnet50": 2,
    "cornet_s": 3
}

# Define a nested dictionary to map combinations of activ_type and stimuli_file_name to offsets
frequency_indices = {
    "alpha": 0,
    "beta": 1,
    "theta": 2,
    "delta": 3,
    "Gamma": 4,

}
combinations = list(itertools.product(model_names, frequency_names))


In [15]:
layers_frequency_sims = np.zeros( (5, 4, 102))
for combination in combinations:
    model_name, frequency_name = combination
    file = os.path.join(similarity_folder, f"{model_name}_Fam_{frequency_name}_main_sim_scores_trained_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)
    layer_similarities, extremum_values = get_layers_similarity(data, networks[model_name])
    i = model_indices.get(model_name)  
    j = frequency_indices.get(frequency_name)  
    layers_frequency_sims[j][i] = np.array(layer_similarities[needed_layers[model_name]][1])

In [7]:
subplot_titles = ["SphereFace", "FaceNet", "ResNet50", "CORnet-S"]


# Create the figure and subplots (5 rows, 4 columns)
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(30, 25))

for row in range(5):
    for col in range(4):
        im, _ = mne.viz.plot_topomap(layers_frequency_sims[row, col], sensors_position, show=False, vlim=(-0.046,0.046),
                                 sphere=0.18, axes=axes[row, col], extrapolate='head')
        axes[4][col].set_xlabel(subplot_titles[col], fontsize=16)
    axes[row][0].set_ylabel(frequency_names[row], fontsize=16)
    fig.colorbar(im, ax=axes[row][3], orientation='vertical')

# Adjust layout
plt.tight_layout()
# Set overall plot title and labels
plt.suptitle('Familiar Input, Trained model, Grad2 plots', fontsize=16, y=1.02)

# Display the plot
plt.show()

In [3]:
model_name = "FaceNet"
stimuli_file = "Fam"
activ_type = "trained"
across_time_folder = os.path.join(similarity_folder, f"across_time/Fam")

In [4]:
results = np.zeros((102, 176))
for i in range(176):
    file = os.path.join(across_time_folder, f"{model_name}_{i:03d}_{stimuli_file}_model_sim_scores_{activ_type}_avg.pkl")
    with open(file, 'rb') as file_data:
        data = pickle.load(file_data)
    for j, sensor in enumerate(channels_grad1):
        results[j][i]=data[f"model {sensor}"]["pearson"][0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

plt.figure(figsize=(16, 12))
# Threshold value for filtering rows
threshold = 0.04

# Find rows with values above the threshold
rows_above_threshold = np.where(np.any(results > threshold, axis=1))[0]

# Find the index of the row that contains the maximum value among the selected rows
max_row_index = rows_above_threshold[np.argmax(np.max(results[rows_above_threshold], axis=1))]

# Calculate the average and standard deviation of the rows above the threshold
average_row = np.mean(results[rows_above_threshold], axis=0)
std_row = np.std(results[rows_above_threshold], axis=0)

# Create a time array for the x-axis (176 * 5 data points, 5 seconds interval)
time_array = np.arange(0, 176 * 5, 5)

# Get the index corresponding to 800 seconds (time index 160)
time_index_800_seconds = 160

# Slice the time array to stop at 800 seconds
time_array_800_seconds = time_array[:time_index_800_seconds + 1]
# Create a new time array with more data points for smoother lines
new_time_array = np.linspace(0, time_index_800_seconds * 5, 1000)

# Interpolate the average and standard deviation data for the new time array
f_average = interp1d(time_array, average_row, kind='cubic')
f_std = interp1d(time_array, std_row, kind='cubic')

# Calculate the interpolated average and standard deviation values
smoothed_average_row = f_average(new_time_array)
smoothed_std_row = f_std(new_time_array)

# Create a bigger figure
plt.figure(figsize=(24, 16))

# Plot the interpolated average of the rows above the threshold
plt.plot(new_time_array, smoothed_average_row, label='Average of Sensors', color='b')


# Interpolate the row with the maximum value for the new time array
f_max_row = interp1d(time_array, results[max_row_index], kind='cubic')
smoothed_max_row = f_max_row(new_time_array)

# Fill the area between one standard deviation below and above the interpolated average line
plt.fill_between(new_time_array, smoothed_average_row - smoothed_std_row, smoothed_average_row + smoothed_std_row, alpha=0.2, color='b')
plt.plot(new_time_array, smoothed_max_row, label=f'MEG2543 sensor (Maximum Sensor Similarity Score)', color='#65aeff')

# Add a vertical line at the position of the maximum value
max_value_time_index = np.argmax(results[rows_above_threshold].max(axis=0))

# Add a vertical line at the position of the maximum value
max_value_time_index = np.argmax(results[max_row_index])
plt.axvline(x=170, color='r', linestyle='--', label=f'Face Recognition F170 Component')

plt.xlabel('Time (ms)')
plt.ylabel('Similarity Scores: Pearson Correlations')
plt.title('Familiar Input, trained FaceNet, Grad2, across time analysis')
plt.legend()
plt.grid(True)
plt.show()

In [1]:
#END